In [141]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np
from Datamanager.DGMT import DGMT
from keras.models import load_model


In [175]:
dgmt = DGMT('../BTCUSDT.csv','time')
dgmt.df.drop(columns=['Unnamed: 0'], inplace=True)
dgmt.change_resolution('1h')
df = dgmt.df
df.dropna(inplace=True)
df

,volume,open,low,high,close,t_plus
time,,,,,,
2017-12-01 00:00:00,275.973987,9837.00,9600.00,9900.03,9685.00,9685.02
2017-12-01 01:00:00,279.578526,9685.02,9550.00,9720.00,9635.00,9634.00
2017-12-01 02:00:00,207.077969,9634.00,9510.00,9720.03,9719.00,9702.00
2017-12-01 03:00:00,205.289027,9702.00,9600.00,9719.98,9698.23,9666.28
2017-12-01 04:00:00,211.509300,9666.28,9617.19,9924.99,9840.00,9832.55
...,...,...,...,...,...,...
2023-06-30 19:00:00,1503.938310,30359.99,30329.24,30474.52,30372.36,30372.37
2023-06-30 20:00:00,1602.035670,30372.37,30366.95,30520.00,30397.54,30397.54
2023-06-30 21:00:00,1011.136510,30397.54,30340.46,30480.00,30479.56,30479.57


In [148]:
# Preprocess the data
scaler = MinMaxScaler()
df['y'] = df.close.shift(-4)
df.dropna(inplace=True)
X = np.array(df['close']).reshape(-1,1)
print(X)
y = np.array(df['y']).reshape(-1,1)
print(y)
X = scaler.fit_transform(X)
y = scaler.fit_transform(y)


train_size = int(len(df) * 0.8)


X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]


[[ 9685.  ]
 [ 9635.  ]
 [ 9719.  ]
 ...
 [30455.8 ]
 [30359.99]
 [30372.36]]
[[ 9840.  ]
 [ 9575.04]
 [ 9449.93]
 ...
 [30479.56]
 [30476.68]
 [30472.  ]]


In [149]:
model = Sequential()
model.add(LSTM(1200, input_shape=(1,1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')

In [150]:
model.fit(X_train, y_train, batch_size=64, epochs=10)

Epoch 1/10
610/610 [==============================] - 21s 33ms/step - loss: 0.0037
Epoch 2/10
610/610 [==============================] - 21s 34ms/step - loss: 5.0533e-05
Epoch 3/10
610/610 [==============================] - 20s 33ms/step - loss: 4.8496e-05
Epoch 4/10
610/610 [==============================] - 15s 24ms/step - loss: 4.8991e-05
Epoch 5/10
610/610 [==============================] - 11s 18ms/step - loss: 4.9564e-05
Epoch 6/10
610/610 [==============================] - 11s 18ms/step - loss: 5.0400e-05
Epoch 7/10
610/610 [==============================] - 11s 18ms/step - loss: 5.2494e-05
Epoch 8/10
610/610 [==============================] - 11s 18ms/step - loss: 5.1296e-05
Epoch 9/10
610/610 [==============================] - 11s 18ms/step - loss: 5.2479e-05
Epoch 10/10
610/610 [==============================] - 11s 18ms/step - loss: 5.3774e-05


In [151]:
test_loss = model.evaluate(X_test, y_test)

305/305 [==============================] - 1s 4ms/step - loss: 2.3269e-05


In [152]:
y_train_pred = model.predict(X_train)
y_test_pred = model.predict(X_test)

305/305 [==============================] - 1s 3ms/step


In [170]:
new_df = scaler.inverse_transform(y_test_pred)

In [178]:
df2 = df.copy()[train_size:-4]


(9758, 9758)

In [179]:
df2

,volume,open,low,high,close,t_plus
time,,,,,,
2022-05-20 05:00:00,1258.50356,30099.66,30025.45,30194.27,30185.39,30185.39
2022-05-20 06:00:00,2104.50897,30185.39,29850.51,30187.81,30005.76,30005.77
2022-05-20 07:00:00,2716.81758,30005.77,29965.21,30362.16,30256.74,30256.74
2022-05-20 08:00:00,2095.49382,30256.74,30151.89,30409.89,30183.99,30183.99
2022-05-20 09:00:00,1803.12593,30183.99,30110.04,30390.00,30301.81,30301.81
...,...,...,...,...,...,...
2023-06-30 15:00:00,3739.87751,30092.49,30000.00,30213.00,30067.72,30067.72
2023-06-30 16:00:00,6623.42035,30067.72,29925.50,30529.39,30400.53,30400.54
2023-06-30 17:00:00,7105.03666,30400.54,30376.59,30748.08,30455.80,30455.79


In [180]:
df2['predicted'] = new_df

In [181]:
df2

,volume,open,low,high,close,t_plus,predicted
time,,,,,,,
2022-05-20 05:00:00,1258.50356,30099.66,30025.45,30194.27,30185.39,30185.39,30367.324219
2022-05-20 06:00:00,2104.50897,30185.39,29850.51,30187.81,30005.76,30005.77,30187.173828
2022-05-20 07:00:00,2716.81758,30005.77,29965.21,30362.16,30256.74,30256.74,30438.884766
2022-05-20 08:00:00,2095.49382,30256.74,30151.89,30409.89,30183.99,30183.99,30365.921875
2022-05-20 09:00:00,1803.12593,30183.99,30110.04,30390.00,30301.81,30301.81,30484.085938
...,...,...,...,...,...,...,...
2023-06-30 15:00:00,3739.87751,30092.49,30000.00,30213.00,30067.72,30067.72,30249.314453
2023-06-30 16:00:00,6623.42035,30067.72,29925.50,30529.39,30400.53,30400.54,30583.085938
2023-06-30 17:00:00,7105.03666,30400.54,30376.59,30748.08,30455.80,30455.79,30638.523438


In [182]:
model.save('new_model.h5')


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [184]:
scaler = MinMaxScaler()
model = load_model('new_model.h5')

X_new = np.array(df2['close']).reshape(-1,1)
scaled_X_new = scaler.fit_transform(X_new)

In [185]:
predictionsssss = model.predict(scaled_X_new)

305/305 [==============================] - 1s 3ms/step


In [187]:
predictions = scaler.inverse_transform(predictionsssss)

In [188]:
predictions

array([[30239.902],
       [30060.418],
       [30311.188],
       ...,
       [30510.057],
       [30414.342],
       [30426.7  ]], dtype=float32)

In [193]:
new_df = pd.DataFrame(X_new)

In [194]:
new_df['predictions'] = predictions
new_df.columns = ['real', 'predictions']

In [220]:
new_df

,real,predictions,diff
0,30185.39,30239.902344,0.180592
1,30005.76,30060.417969,0.182158
2,30256.74,30311.187500,0.179952
3,30183.99,30238.503906,0.180605
4,30301.81,30356.214844,0.179543
...,...,...,...
9753,30067.72,30122.326172,0.181611
9754,30400.53,30454.843750,0.178661
9755,30455.80,30510.056641,0.178149
9756,30359.99,30414.341797,0.179024


In [209]:
new_df['diff'] = ((new_df['predictions'] / new_df['real']) - 1) * 100

In [223]:
new_df.to_csv('model_analysis.csv')
